## Basic Chatbot - Connecting with LLM
Let's start with the simplest possible agent - a basic chatbot that connects to an LLM.

In [1]:
from phi.agent import Agent
from phi.model.openai import OpenAIChat

# Create a basic agent
agent = Agent(
    model=OpenAIChat(id="gpt-4o-mini"),
    description="A helpful assistant"
)

# Simple conversation
response = agent.run("Hello! What can you help me with?")
print(response.content)

# Continue the conversation
response = agent.run("Tell me a joke about programming")
print(response.content)

Hello! I can assist you with a wide range of topics, including:

- Answering questions about various subjects (science, history, technology, etc.)
- Providing information or summaries on specific topics
- Offering writing assistance, such as editing or generating text
- Helping with language learning or translations
- Providing tips for studying or managing time effectively
- Giving advice on problem-solving or decision-making
- Offering recommendations for books, movies, or other media

Let me know what you need help with!
Why do programmers prefer dark mode?

Because light attracts bugs!


## System Messages and Communication
Now let's add system messages to give our agent specific instructions and personality.

In [2]:
from phi.agent import Agent
from phi.model.openai import OpenAIChat

# Agent with detailed system message
agent = Agent(
    model=OpenAIChat(id="gpt-4o-mini"),
    description="You are a Python programming tutor",
    instructions=[
        "You specialize in teaching Python programming to beginners",
        "Always provide code examples when explaining concepts",
        "Be encouraging and patient",
        "If asked about other programming languages, gently redirect to Python",
        "End each response with a relevant Python tip"
    ],
    show_tool_calls=True,
    markdown=True
)

# Test the agent's behavior
print("=== Python Tutor Agent ===")
response = agent.run("How do I create a list in Python?")
print(response.content)

print("\n" + "="*50 + "\n")

response = agent.run("What about JavaScript arrays?")
print(response.content)

=== Python Tutor Agent ===
Creating a list in Python is quite simple! A list is a versatile data structure that can hold multiple items, which can be of different types, all under one variable name. You can create a list by wrapping your elements in square brackets `[]`.

Here's a basic example showing how to create a list:

```python
# Creating a list of fruits
fruits = ['apple', 'banana', 'cherry']

print(fruits)
```

In this example, we created a list called `fruits` that contains three strings: `"apple"`, `"banana"`, and `"cherry"`. When you print the list, it will display the contents.

### Adding to a List
You can also add items to your list using the `append()` method like this:

```python
# Adding an item to the list
fruits.append('orange')

print(fruits)
```

After running this code, the list will now include "orange", and look like this:

```
['apple', 'banana', 'cherry', 'orange']
```

### Tip
Always remember that Python lists can hold items of different data types, so you c

## Connecting Tools and Functions
Agents become powerful when they can use tools. Let's create an agent with various tools.

In [3]:
from phi.agent import Agent
from phi.model.openai import OpenAIChat
from phi.tools.duckduckgo import DuckDuckGo
from phi.tools.python import PythonTools
import json
from datetime import datetime

# Custom tool function
def get_current_time() -> str:
    """Get the current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def calculate_compound_interest(principal: float, rate: float, time: int, compound_frequency: int = 1) -> dict:
    """
    Calculate compound interest.
    
    Args:
        principal: Initial amount
        rate: Annual interest rate (as decimal, e.g., 0.05 for 5%)
        time: Time in years
        compound_frequency: How many times interest compounds per year
    
    Returns:
        Dictionary with calculation results
    """
    amount = principal * (1 + rate/compound_frequency) ** (compound_frequency * time)
    interest = amount - principal
    
    return {
        "principal": principal,
        "rate": rate * 100,  # Convert to percentage
        "time": time,
        "compound_frequency": compound_frequency,
        "final_amount": round(amount, 2),
        "interest_earned": round(interest, 2)
    }

# Agent with multiple tools
tool_agent = Agent(
    model=OpenAIChat(id="gpt-4o-mini"),
    description="A versatile assistant with access to web search, Python execution, and financial calculations",
    instructions=[
        "You can search the web for current information",
        "You can execute Python code to solve problems",
        "You can calculate compound interest for financial questions",
        "You can get the current time",
        "Always explain what tools you're using and why"
    ],
    tools=[
        DuckDuckGo(),
        PythonTools(),
        get_current_time,
        calculate_compound_interest
    ],
    show_tool_calls=True,
    markdown=True
)

# Test different tool capabilities
print("=== Tool-Enabled Agent ===")

response = tool_agent.run("What is RAG system mean?")
print("Duckduck query :", response.content)
print("\n" + "="*50 + "\n")

# Test time function
response = tool_agent.run("What time is it?")
print("Time Query:", response.content)
print("\n" + "="*50 + "\n")

# Test Python execution
response = tool_agent.run("Create a list of the first 10 prime numbers and show me the code")
print("Python Execution:", response.content)
print("\n" + "="*50 + "\n")

# Test compound interest calculation
#response = tool_agent.run("If I invest $10,000 at 7% annual interest compounded monthly for 5 years, how much will I have?")
#print("Financial Calculation:", response.content)

=== Tool-Enabled Agent ===
Duckduck query : The RAG system, which stands for Red, Amber, Green, is a traffic light coding system used to assess the status of projects, tasks, or performance indicators. Here's a breakdown of its meaning:

- **Red:** Indicates a serious issue or that the project or task is significantly off track. Immediate action is needed to address the problems.

- **Amber (or Yellow):** Signifies that there are some issues or risks present, but they are manageable. It usually requires monitoring or adjustments to avoid escalating into a red status.

- **Green:** Represents that everything is proceeding smoothly as planned. There are no significant issues, and the project or task is on track to meet its goals.

This system is commonly used in project management, performance monitoring, and reporting to provide a quick visual representation of the status of various elements.


Time Query: 
Running:
 - get_current_time()

The current time is **09:32 AM** on **August 10,

WARNING  PythonTools can run arbitrary code, please provide human supervision.

INFO     Saved: /Users/utsabchakraborty/Documents/Agentic_Class_Live_Phidata /first_10_primes.py

INFO     Running /Users/utsabchakraborty/Documents/Agentic_Class_Live_Phidata /first_10_primes.py

Python Execution: 
Running:
 - save_to_file_and_run(file_name=first_10_primes.py, code=..., variable_to_return=primes, overwrite=True)

The first 10 prime numbers are:

\[ 2, 3, 5, 7, 11, 13, 17, 19, 23, 29 \]

### Code Explanation

Here’s the Python code that generates the first 10 prime numbers:

```python
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

primes = []
num = 2
while len(primes) < 10:
    if is_prime(num):
        primes.append(num)
    num += 1

primes
```

- **Function `is_prime(n)`**: Checks if a number \( n \) is prime.
- **List `primes`**: Stores the prime numbers.
- **While loop**: Continues until we have 10 prime numbers.
- **The condition inside the loop**: Evaluates whether the current number `num` is prime and adds it to the list if true. 

This code effectively generates the first 10 primes through iteration and condition checking.




# End-to-End Single Agent Application
Let's build a complete personal finance advisor application.

In [4]:
from phi.agent import Agent
from phi.model.openai import OpenAIChat

def calculate_savings_rate(monthly_income: float, monthly_expenses: float) -> str:
    """
    Calculate savings rate and provide financial advice.
    
    Args:
        monthly_income: Monthly gross income
        monthly_expenses: Total monthly expenses
    
    Returns:
        String with savings rate analysis and advice
    """
    remaining = monthly_income - monthly_expenses
    savings_rate = (remaining / monthly_income) * 100 if monthly_income > 0 else 0
    
    if savings_rate >= 20:
        health = "Excellent"
        advice = "You're saving well above the recommended 20%. Consider investing excess savings."
    elif savings_rate >= 10:
        health = "Good"
        advice = "You're doing well, but try to increase savings to 20% if possible."
    elif savings_rate >= 0:
        health = "Needs Improvement"
        advice = "Try to reduce expenses or increase income to save at least 10-20%."
    else:
        health = "Critical"
        advice = "You're spending more than you earn. Immediate budget adjustments needed."
    
    result = f"""
BUDGET ANALYSIS:
Monthly Income: ${monthly_income:,.2f}
Monthly Expenses: ${monthly_expenses:,.2f}
Remaining Money: ${remaining:,.2f}
Savings Rate: {savings_rate:.1f}%
Financial Health: {health}

ADVICE: {advice}
"""
    
    return result.strip()

def retirement_calculator(current_age: int, retirement_age: int, monthly_savings: float) -> str:
    """
    Simple retirement calculator with 7% annual return assumption.
    
    Args:
        current_age: Current age
        retirement_age: Target retirement age  
        monthly_savings: Monthly savings amount
    
    Returns:
        String with retirement projection
    """
    years_to_retirement = retirement_age - current_age
    months_to_retirement = years_to_retirement * 12
    
    # Simple calculation assuming 7% annual return (0.58% monthly)
    monthly_return = 0.0058
    
    if monthly_return > 0:
        total_savings = monthly_savings * (((1 + monthly_return) ** months_to_retirement - 1) / monthly_return)
    else:
        total_savings = monthly_savings * months_to_retirement
    
    annual_income_4percent = total_savings * 0.04
    monthly_income_retirement = annual_income_4percent / 12
    
    result = f"""
RETIREMENT PROJECTION:
Current Age: {current_age}
Retirement Age: {retirement_age}
Years to Retirement: {years_to_retirement}
Monthly Savings: ${monthly_savings:,.2f}

PROJECTIONS (assuming 7% annual return):
Total Retirement Savings: ${total_savings:,.2f}
Annual Retirement Income (4% rule): ${annual_income_4percent:,.2f}
Monthly Retirement Income: ${monthly_income_retirement:,.2f}
"""
    
    return result.strip()

# Create a simple finance agent
simple_finance_agent = Agent(
    model=OpenAIChat(id="gpt-4o-mini"),
    description="Personal finance advisor that helps with budgeting and retirement planning",
    instructions=[
        "You are a helpful personal finance advisor",
        "Use the provided tools to calculate savings rates and retirement projections", 
        "Provide clear, actionable financial advice",
        "Ask for specific numbers when you need them to perform calculations",
        "Explain financial concepts in simple terms"
    ],
    tools=[
        calculate_savings_rate,
        retirement_calculator
    ],
    show_tool_calls=True,
    markdown=True
)

# Test the simple agent
if __name__ == "__main__":
    print("=== Simple Finance Agent Test ===\n")
    
    # Test 1: Budget Analysis
    print("💰 BUDGET ANALYSIS")
    print("-" * 30)
    
    response1 = simple_finance_agent.run("""
    Hi! I am 38 years old male , planning to retire at 60. I make 50,000 Rs per month and my total expenses are 30,000 Rs per month. 
    Can you analyze my budget and tell me how I'm doing financially?
    """)
    
    print(response1.content)
    print("\n" + "="*50 + "\n")
    
    # Test 2: Retirement Planning
    #print("🏖️ RETIREMENT PLANNING")
    #print("-" * 30)
    
    #response2 = simple_finance_agent.run("""
    #I'm 30 years old and want to retire at 65. Based on my budget above, 
    #if I can save $800 per month, what will my retirement look like?
    #""")
    
    #print(response2.content)
    #print("\n" + "="*50 + "\n")
    
    # Test 3: What-if scenario
    print("❓ WHAT-IF SCENARIO")
    print("-" * 30)
    
    response3 = simple_finance_agent.run("""
    What if I could reduce my expenses to 25000 Rs per month and increase my monthly 
    savings by 5000 Rs more ? How would that change my retirement projection?
    """)
    
    print(response3.content)

=== Simple Finance Agent Test ===

💰 BUDGET ANALYSIS
------------------------------

Running:
 - calculate_savings_rate(monthly_income=50000, monthly_expenses=30000)
 - retirement_calculator(current_age=38, retirement_age=60, monthly_savings=20000)

### Budget Analysis
- **Monthly Income:** ₹50,000
- **Monthly Expenses:** ₹30,000
- **Remaining Money:** ₹20,000
- **Savings Rate:** 40.0%
- **Financial Health:** Excellent

### Financial Advice
You're doing really well with your budgeting! Saving 40% of your income is significantly above the recommended rate of 20%. Consider investing your excess savings to grow your wealth even further, which can help with future financial goals and inflation.

### Retirement Projection
- **Current Age:** 38
- **Retirement Age:** 60
- **Years to Retirement:** 22
- **Monthly Savings:** ₹20,000

#### Projections (assuming a 7% annual return):
- **Total Retirement Savings:** ₹12,425,222.86
- **Annual Retirement Income (4% rule):** ₹497,008.91
- **Monthly Ret

# Multi-Agent Basic Communication
Now let's create multiple agents that can work together. We'll build a content creation team.

In [5]:
from phi.agent import Agent
from phi.model.openai import OpenAIChat
from phi.tools.duckduckgo import DuckDuckGo
from phi.tools.python import PythonTools

# Research Agent
researcher = Agent(
    name="Content Researcher",
    model=OpenAIChat(id="gpt-4o-mini"),
    description="Research specialist who gathers information on topics",
    instructions=[
        "You are responsible for researching topics thoroughly",
        "Use web search to find current and accurate information",
        "Organize your findings in a clear, structured way",
        "Focus on credible sources and recent information",
        "Provide key statistics, trends, and expert opinions"
    ],
    tools=[DuckDuckGo()],
    show_tool_calls=True,
    markdown=True
)

# Writer Agent
writer = Agent(
    name="Content Writer",
    model=OpenAIChat(id="gpt-4o-mini"),
    description="Professional content writer who creates engaging articles",
    instructions=[
        "You create well-structured, engaging content",
        "Write in a clear, professional tone",
        "Include compelling headlines and subheadings",
        "Use the research provided to create accurate content",
        "Make complex topics accessible to general audiences",
        "Include practical examples and actionable insights"
    ],
    markdown=True
)

# Editor Agent
editor = Agent(
    name="Content Editor",
    model=OpenAIChat(id="gpt-4o-mini"),
    description="Editorial specialist who reviews and improves content",
    instructions=[
        "You review content for clarity, flow, and engagement",
        "Check for grammar, spelling, and style consistency",
        "Suggest improvements for structure and readability",
        "Ensure content meets professional standards",
        "Provide specific, actionable feedback"
    ],
    markdown=True
)

def content_creation_workflow(topic: str) -> str:
    """
    Orchestrates the content creation process using multiple agents.
    """
    print(f"=== Content Creation Workflow: {topic} ===\n")
    
    # Step 1: Research
    print("🔍 RESEARCH PHASE")
    print("-" * 30)
    research_prompt = f"""
    Please research the topic: "{topic}"
    
    Focus on:
    1. Current trends and developments
    2. Key statistics and data points  
    3. Expert opinions and insights
    4. Practical applications or implications
    5. Common challenges or misconceptions
    
    Provide a comprehensive research summary.
    """
    
    research_result = researcher.run(research_prompt)
    print("Research completed:")
    print(research_result.content)
    print("\n" + "="*50 + "\n")
    
    # Step 2: Writing
    print("✍️ WRITING PHASE")
    print("-" * 30)
    writing_prompt = f"""
    Based on the following research, write a comprehensive article about "{topic}":

    RESEARCH FINDINGS:
    {research_result.content}
    
    Please create:
    1. An engaging title
    2. A compelling introduction
    3. Well-organized main content with subheadings
    4. Practical examples or case studies
    5. A strong conclusion with key takeaways
    
    Target length: 800-1000 words
    Audience: General public with interest in the topic
    """
    
    article_draft = writer.run(writing_prompt)
    print("Draft article completed:")
    print(article_draft.content)
    print("\n" + "="*50 + "\n")
    
    # Step 3: Editing
    print("📝 EDITING PHASE")
    print("-" * 30)
    editing_prompt = f"""
    Please review and edit the following article:

    {article_draft.content}
    
    Provide:
    1. Overall assessment of the article
    2. Specific suggestions for improvement
    3. Final edited version
    
    Focus on clarity, engagement, and professional quality.
    """
    
    final_result = editor.run(editing_prompt)
    print("Editorial review completed:")
    print(final_result.content)
    
    return final_result.content

# Example usage
topic = "The Impact of GenAI on Indian IT Industry in not more than 200 words"
final_article = content_creation_workflow(topic)

=== Content Creation Workflow: The Impact of GenAI on Indian IT Industry in not more than 200 words ===

🔍 RESEARCH PHASE
------------------------------


/Users/utsabchakraborty/Documents/Agentic_Class_Live_Phidata /phidata-env/lib/python3.12/site-packages/phi/tools/duckduckgo.py:65: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  ddgs = DDGS(
/Users/utsabchakraborty/Documents/Agentic_Class_Live_Phidata /phidata-env/lib/python3.12/site-packages/phi/tools/duckduckgo.py:86: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  ddgs = DDGS(


Research completed:

Running:
 - duckduckgo_search(query=Impact of GenAI on Indian IT industry 2023, max_results=5)


Running:
 - duckduckgo_news(query=Impact of GenAI on Indian IT industry 2023, max_results=5)

## The Impact of GenAI on the Indian IT Industry

### Current Trends and Developments
The Indian IT industry is witnessing transformative shifts driven by generative AI (GenAI). Notably, the sector has attracted over $750 million in funding for GenAI startups since 2023, indicating a robust wave of innovation.

### Key Statistics and Data Points
- India's GenAI market is anticipated to surge from $1.1 billion in 2023 to $17 billion by 2030, reflecting a compound annual growth rate (CAGR) of 48%.
- GenAI could enhance productivity by up to 45% over the next five years, significantly impacting the $254 billion IT sector.

### Expert Opinions and Insights
Industry experts warn of a paradoxical outcome: while GenAI fosters innovation, it could lead to approximately 125,000 job loss

# Complex Multi-Agent with Memory
Finally, let's build a sophisticated customer service system with memory and advanced coordination.

In [ ]:
from phi.agent import Agent
from phi.model.openai import OpenAIChat
from phi.tools.duckduckgo import DuckDuckGo
from phi.storage.agent.postgres import PgAgentStorage
from phi.memory import AgentMemory
from datetime import datetime
from typing import Dict, List, Optional
import json
from phi.tools import tool


# For this example, we'll use in-memory storage instead of Postgres
# In production, you'd use actual database storage

class CustomerDatabase:
    """Simulated customer database"""
    
    def __init__(self):
        self.customers = {
            "john.doe@email.com": {
                "name": "John Doe",
                "account_type": "Premium",
                "join_date": "2023-01-15",
                "total_orders": 12,
                "last_order": "2024-07-20",
                "support_tickets": 3,
                "satisfaction_score": 4.2
            },
            "jane.smith@email.com": {
                "name": "Jane Smith", 
                "account_type": "Standard",
                "join_date": "2023-06-10",
                "total_orders": 5,
                "last_order": "2024-07-28",
                "support_tickets": 1,
                "satisfaction_score": 4.8
            }
        }
    @tool   
    def get_customer(self, email: str) -> Optional[Dict]:
        """Get customer information by email"""
        return self.customers.get(email)
    
    def get_customer(self, email: str) -> Optional[Dict]:
        """Get customer information by email"""
        return self.customers.get(email)
    
    @tool
    def update_customer(self, email: str, updates: Dict) -> bool:
        """Update customer information"""
        if email in self.customers:
            self.customers[email].update(updates)
            return True
        return False
    
    def update_customer(self, email: str, updates: Dict) -> bool:
        """Update customer information"""
        if email in self.customers:
            self.customers[email].update(updates)
            return True
        return False


class TicketSystem:
    """Simulated ticket management system"""
    
    def __init__(self):
        self.tickets = {}
        self.ticket_counter = 1000
    
    @tool
    def create_ticket(self, customer_email: str, issue_type: str, description: str, priority: str = "Medium") -> str:
        """Create a new support ticket"""
        ticket_id = f"TICKET-{self.ticket_counter}"
        self.ticket_counter += 1
        
        self.tickets[ticket_id] = {
            "id": ticket_id,
            "customer_email": customer_email,
            "issue_type": issue_type,
            "description": description,
            "priority": priority,
            "status": "Open",
            "created_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "assigned_agent": None,
            "resolution": None
        }
        
        return ticket_id
    def create_ticket(self, customer_email: str, issue_type: str, description: str, priority: str = "Medium") -> str:
        """Create a new support ticket"""
        ticket_id = f"TICKET-{self.ticket_counter}"
        self.ticket_counter += 1
        
        self.tickets[ticket_id] = {
            "id": ticket_id,
            "customer_email": customer_email,
            "issue_type": issue_type,
            "description": description,
            "priority": priority,
            "status": "Open",
            "created_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "assigned_agent": None,
            "resolution": None
        }
        
        return ticket_id
    @tool
    def get_ticket(self, ticket_id: str) -> Optional[Dict]:
        """Get ticket information"""
        return self.tickets.get(ticket_id)
    
    def get_ticket(self, ticket_id: str) -> Optional[Dict]:
        """Get ticket information"""
        return self.tickets.get(ticket_id)
    @tool
    def update_ticket(self, ticket_id: str, updates: Dict) -> bool:
        """Update ticket information"""
        if ticket_id in self.tickets:
            self.tickets[ticket_id].update(updates)
            return True
        return False
    def update_ticket(self, ticket_id: str, updates: Dict) -> bool:
        """Update ticket information"""
        if ticket_id in self.tickets:
            self.tickets[ticket_id].update(updates)
            return True
        return False

# Initialize systems
customer_db = CustomerDatabase()
ticket_system = TicketSystem()

# Customer Service Agent
customer_service_agent = Agent(
    name="Customer Service Representative",
    model=OpenAIChat(id="gpt-4o-mini"),
    description="Primary customer service agent handling inquiries and issues",
    instructions=[
        "You are a friendly and professional customer service representative",
        "Always greet customers warmly and get their information",
        "Listen carefully to customer concerns and ask clarifying questions",
        "Use available tools to look up customer information and create tickets",
        "Escalate complex technical issues to the technical support agent",
        "Escalate billing/account issues to the billing specialist",
        "Always follow up to ensure customer satisfaction",
        "Remember previous interactions with customers"
    ],
    tools=[
        customer_db.get_customer,
        customer_db.update_customer,
        ticket_system.create_ticket,
        ticket_system.get_ticket
    ],
    # In-memory conversation storage for this example
    storage=None,  # Would use PgAgentStorage("postgresql://...") in production
    add_history_to_messages=True,
    num_history_responses=5,
    show_tool_calls=True,
    markdown=True
)

# Technical Support Agent
tech_support_agent = Agent(
    name="Technical Support Specialist",
    model=OpenAIChat(id="gpt-4o-mini"),
    description="Technical expert handling complex technical issues",
    instructions=[
        "You are a technical support specialist with deep product knowledge",
        "Handle complex technical troubleshooting and bug reports",
        "Provide step-by-step technical solutions",
        "Use web search for latest technical documentation when needed",
        "Create detailed technical tickets with reproduction steps",
        "Escalate to engineering team if necessary",
        "Update ticket status and resolutions"
    ],
    tools=[
        DuckDuckGo(),
        ticket_system.get_ticket,
        ticket_system.update_ticket,
        customer_db.get_customer
    ],
    add_history_to_messages=True,
    num_history_responses=3,
    show_tool_calls=True,
    markdown=True
)

# Billing Specialist Agent
billing_agent = Agent(
    name="Billing Specialist",
    model=OpenAIChat(id="gpt-4o-mini"),
    description="Billing and account specialist handling payment and subscription issues",
    instructions=[
        "You are a billing specialist handling payment, refund, and subscription issues",
        "Access customer account information to resolve billing questions",
        "Process refund requests according to company policy",
        "Handle subscription upgrades, downgrades, and cancellations",
        "Explain billing charges and payment methods clearly",
        "Create billing-related tickets for complex issues",
        "Ensure all billing actions are properly documented"
    ],
    tools=[
        customer_db.get_customer,
        customer_db.update_customer,
        ticket_system.create_ticket,
        ticket_system.get_ticket,
        ticket_system.update_ticket
    ],
    add_history_to_messages=True,
    num_history_responses=3,
    show_tool_calls=True,
    markdown=True
)

class CustomerServiceOrchestrator:
    """Orchestrates customer service interactions across multiple agents"""
    
    def __init__(self):
        self.active_sessions = {}
    
    def route_customer_inquiry(self, customer_email: str, inquiry: str, session_id: str = None) -> str:
        """
        Routes customer inquiry to appropriate agent and manages the conversation flow
        """
        if not session_id:
            session_id = f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        
        # Determine which agent should handle this inquiry
        inquiry_lower = inquiry.lower()
        
        if any(keyword in inquiry_lower for keyword in ['technical', 'bug', 'error', 'not working', 'broken', 'crash']):
            agent = tech_support_agent
            agent_type = "technical"
        elif any(keyword in inquiry_lower for keyword in ['billing', 'payment', 'refund', 'charge', 'subscription', 'upgrade', 'cancel']):
            agent = billing_agent
            agent_type = "billing"
        else:
            agent = customer_service_agent
            agent_type = "general"
        
        # Prepare context-aware prompt
        customer_context = customer_db.get_customer(customer_email)
        context_info = ""
        if customer_context:
            context_info = f"""
CUSTOMER INFORMATION:
- Name: {customer_context['name']}
- Account Type: {customer_context['account_type']}
- Member Since: {customer_context['join_date']}
- Total Orders: {customer_context['total_orders']}
- Last Order: {customer_context['last_order']}
- Previous Support Tickets: {customer_context['support_tickets']}
- Satisfaction Score: {customer_context['satisfaction_score']}/5.0
"""
        
        full_prompt = f"""
{context_info}

CUSTOMER INQUIRY:
Customer Email: {customer_email}
Session ID: {session_id}
Message: {inquiry}

Please help this customer with their inquiry. Use available tools as needed.
"""
        
        # Get response from appropriate agent
        response = agent.run(full_prompt)
        
        # Store session information
        self.active_sessions[session_id] = {
            "customer_email": customer_email,
            "agent_type": agent_type,
            "start_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        
        return response.content
    
    def continue_conversation(self, session_id: str, message: str) -> str:
        """Continue an existing conversation"""
        if session_id not in self.active_sessions:
            return "Session not found. Please start a new inquiry."
        
        session_info = self.active_sessions[session_id]
        agent_type = session_info["agent_type"]
        
        # Route to the same agent that started the conversation
        if agent_type == "technical":
            agent = tech_support_agent
        elif agent_type == "billing":
            agent = billing_agent
        else:
            agent = customer_service_agent
        
        response = agent.run(f"Customer follow-up message: {message}")
        return response.content

# Example usage
print("=== Advanced Multi-Agent Customer Service System ===\n")

orchestrator = CustomerServiceOrchestrator()

# Simulate customer interactions
print("🔹 Customer 1: Technical Issue")
print("-" * 40)
response1 = orchestrator.route_customer_inquiry(
    customer_email="john.doe@email.com",
    inquiry="Hi, I'm having trouble with the app crashing every time I try to upload a file. This has been happening for the past 2 days.",
    session_id="session_001"
)
print("Agent Response:")
print(response1)
print("\n" + "="*60 + "\n")

print("🔹 Customer 2: Billing Issue")
print("-" * 40)
response2 = orchestrator.route_customer_inquiry(
    customer_email="jane.smith@email.com", 
    inquiry="I was charged twice for my subscription this month. Can you please help me get a refund for the duplicate charge?",
    session_id="session_002"
)
print("Agent Response:")
print(response2)
print("\n" + "="*60 + "\n")

print("🔹 Follow-up from Customer 1")
print("-" * 40)
followup = orchestrator.continue_conversation(
    session_id="session_001",
    message="I tried the troubleshooting steps but the app is still crashing. The error message says 'Memory allocation failed'."
)
print("Agent Follow-up Response:")
print(followup)

=== Advanced Multi-Agent Customer Service System ===

🔹 Customer 1: Technical Issue
----------------------------------------


TypeError: 'Function' object is not callable